# Data Preparation

Authored by [JumpThanawut](https://github.com/orgs/Datatouille/people/JumpThanawut); Edited by [cstorm125](https://github.com/cstorm125/)

The `snaplogic_snap_recommendation` training set contains 25,422 pipelines used internally at SnapLogic, those pipelines were built before Jan 1, 2019. The test set contains 7,614 pipelines which have been built in 2019. Each pipeline contains a series of **snaps**--a component with data mainpulation codes. No pipeline in this dataset belongs to SnapLogic customers.

Raw data is `snaplogic_pipeline_public.json`. The resulting `snaplogic_pipeline_segment.csv` contains 147,304 rows of 5-snap segments (4 previous snaps and 1 target snap) and the following columns:

* `date` - date created
* `org` - team identifier
* `prev_snap_1-4` - the past 1-4 snaps
* `project` - project identifier
* `target_snap` - snap to predict
* `user` - user identifier

The task is predict `target_snap` based on all other variables. We divide the dataset into train-validation-test splits at 70/10/20 ratio in chronological order. Performance metrics are top-1 and top-5 accuracy.

In [1]:
# #uncomment if you are running from google colab
# !wget https://github.com/Datatouille/snaplogic_snap_recommendation/archive/master.zip; unzip master
# !mv snaplogic_snap_recommendation-master/* .
# !ls

In [2]:
import numpy as np
import pandas as pd
import json
from collections import Counter

## Pipeline Format

One pipeline consists of the following JSON format.

## Read JSON

In [3]:
with open("./dataset/snaplogic_pipeline_public_train.json", "r") as pipeline_dataset_file:
    pipe_list = json.load(pipeline_dataset_file)

# Print dataset size.
print(f"Pipeline: {len(pipe_list)}")

# Print first and last record.
print("----- First Record -----")
print(json.dumps(pipe_list[0], indent=2))
print("----- Last Record -----")
print(json.dumps(pipe_list[-1], indent=2))

Pipeline: 25422
----- First Record -----
{
  "date": "2014-12-05",
  "org": "1",
  "project": "3",
  "user": "230",
  "snap_map": {
    "1": "287",
    "2": "158",
    "3": "185"
  },
  "link_map": {
    "1": [
      "1",
      "3"
    ],
    "2": [
      "3",
      "2"
    ]
  }
}
----- Last Record -----
{
  "date": "2018-12-26",
  "org": "3",
  "project": "471",
  "user": "146",
  "snap_map": {
    "1": "51",
    "2": "174"
  },
  "link_map": {
    "1": [
      "1",
      "2"
    ]
  }
}


## Descriptive Statistics

Here are some descriptive statistics about the pipelines. We found that each pipeline contains about 5.818464 Snaps. Therefore, we decided to divide all pipelines into 5-snap segments for the recommendation dataset. Note that the real average number of Snaps per pipeline is about 12 based on the full dataset including our customers' pipelines.

In [4]:
# Get some statistics of pipelines.
date_counter = Counter()
org_counter = Counter()
project_counter = Counter()
user_counter = Counter()
snap_type_counter = Counter()

for pipe in pipe_list:
    
    date = pipe["date"]
    org = pipe["org"]
    project = pipe["project"]
    user = pipe["user"]
    snap_map = pipe["snap_map"]
    link_map = pipe["link_map"]
    
    date_counter[date] += 1
    org_counter[org] += 1
    project_counter[project] += 1
    user_counter[user] += 1
    
    for snap_id in snap_map:
        snap_type = snap_map[snap_id]
        snap_type_counter[snap_type] += 1
        
num_snap = sum([len(pipe["snap_map"]) for pipe in pipe_list])
avg_num_snap_in_pipe = num_snap / len(pipe_list)
sd_num_snap = np.std(np.array([len(pipe["snap_map"]) for pipe in pipe_list]))

num_link = sum([len(pipe["link_map"]) for pipe in pipe_list])
avg_num_link_in_pipe = num_link / len(pipe_list)
sd_num_link_in_pipe = np.std(np.array([len(pipe["link_map"]) for pipe in pipe_list]))

print("Pipeline: {:d}".format(len(pipe_list)))
print("Snap: {:d}".format(num_snap))
print("Start Date: {:s}".format(min(date_counter.keys())))
print("Org: {:d}".format(len(org_counter)))
print("Project: {:d}".format(len(project_counter)))
print("User: {:d}".format(len(user_counter)))
print("Snap Type: {:d}".format(len(snap_type_counter)))
print("Average Number of Snaps in a Pipeline: {:f}".format(avg_num_snap_in_pipe))
print("SD Number of Snaps in a Pipeline: {:f}".format(sd_num_snap))
print("Average Number of Links in a Pipeline: {:f}".format(avg_num_link_in_pipe))
print("SD Number of Links in a Pipeline: {:f}".format(sd_num_link_in_pipe))

Pipeline: 25422
Snap: 147917
Start Date: 2014-12-05
Org: 4
Project: 516
User: 220
Snap Type: 485
Average Number of Snaps in a Pipeline: 5.818464
SD Number of Snaps in a Pipeline: 5.040545
Average Number of Links in a Pipeline: 4.675989
SD Number of Links in a Pipeline: 5.131295


In [5]:
# Explore distribution of pipelines among orgs. SnapLogic has 4 internal orgs on production.
print("----- Org -----")
print(json.dumps(org_counter, indent = 2, sort_keys = True))

----- Org -----
{
  "1": 2489,
  "2": 288,
  "3": 22481,
  "4": 164
}


## Extract Segments

We extract all the segments and save as CSV.

In [6]:
# Extract segment from pipelines.
segment_len = 5
segment_list = []

# Represent pipeline which is DAG (directed acyclic graph) as adjacency matrix.
def to_adj_matrix(pipe):
    # adj_matrix = {"<snap_id>": [<upstream_snap>, ...], ...}
    adj_matrix = {}
    snap_map = pipe["snap_map"]
    link_map = pipe["link_map"]
    
    for link_id in link_map:
        src_id, dst_id = link_map[link_id]
        if dst_id not in adj_matrix:
            adj_matrix[dst_id] = []
        if src_id not in adj_matrix[dst_id]:
            adj_matrix[dst_id].append(src_id)
    
    return adj_matrix

# Get segment (consecutive Snaps) from the specified Snap. The snap_id is the id of Snap which will be the last Snap in the segment.
def get_segment(adj_matrix, snap_id, segment_len):
    if segment_len == 1:
        return [snap_id]
    else:
        upst_snap_list = adj_matrix.get(snap_id, [])
        if len(upst_snap_list) > 0:
            child_segment = get_segment(adj_matrix, upst_snap_list[0], segment_len-1)
            child_segment.append(snap_id)
            return child_segment
        else:
            return [snap_id]

# Extract segments from pipelines.
for pipe in pipe_list:
    adj_matrix = to_adj_matrix(pipe)
    snap_map = pipe["snap_map"]
    for snap_id in snap_map:
        segment = get_segment(adj_matrix, snap_id, segment_len)
        
        segment_map = {
            "date": pipe["date"],
            "org": pipe["org"],
            "project": pipe["project"],
            "user": pipe["user"],
            "target_snap": snap_map[snap_id]
        }
        
        for i in range(1, segment_len):
            if i < len(segment):
                snap_type = snap_map[segment[-i-1]]
            else:
                snap_type = ""
            segment_map["prev_snap_{:d}".format(i)] = snap_type
        
        segment_list.append(segment_map)
print("Segment: {:d}".format(len(segment_list)))

Segment: 147917


In [7]:
# Visualize the dataset as table.
df = pd.DataFrame(segment_list)
df.head()

,date,org,prev_snap_1,prev_snap_2,prev_snap_3,prev_snap_4,project,target_snap,user
0,2014-12-05,1,,,,,3,287,230
1,2014-12-05,1,185,287,,,3,158,230
2,2014-12-05,1,287,,,,3,185,230
3,2015-02-10,1,158,194,470,,114,287,230
4,2015-02-10,1,470,,,,114,194,230


In [8]:
# Save dataset to file in JSON format.
with open("dataset/snaplogic_pipeline_segment_train.json", "w") as segment_file:
    json.dump(segment_list, segment_file)

#impute 0
df[df==''] = 0

# Save dataset to file in CSV format.
df.to_csv("dataset/snaplogic_pipeline_segment_train.csv", index=False, encoding="utf-8",
          columns=["date","org","project", "user", 
                "prev_snap_4", "prev_snap_3", "prev_snap_2", "prev_snap_1", "target_snap"])

## Train-Valid-Test Splits

In [9]:
train_ends = int(df.shape[0] * 0.7)
valid_ends = int(df.shape[0] * 0.8)

train_df = df.iloc[:train_ends,:].drop(["date"], axis=1).sample(frac=1)
valid_df = df.iloc[train_ends:valid_ends,:].drop(["date"], axis=1).sample(frac=1)
test_df = df.iloc[valid_ends:,:].drop(["date"], axis=1).sample(frac=1)

train_df.shape, valid_df.shape, test_df.shape

((103541, 8), (14792, 8), (29584, 8))

In [10]:
train_df.to_csv('dataset/train_df.csv',index=False)
valid_df.to_csv('dataset/valid_df.csv',index=False)
test_df.to_csv('dataset/test_df.csv',index=False)